Process results obtained via running command:
```
bash -i run_main.sh
```

In [1]:
SEED_QTY=3
OUTPUT_PREF='results/output_res_'

In [2]:
res_keys = ['1gpu'] + \
        [f'accum_steps_{k}' for k in [1, 2, 4, 8, 16]] + \
        [f'nosync_steps_{k}' for k in [1, 2, 4, 8, 16]]
res_dict = {k : [] for k in res_keys}
res_dict_str = {k : [] for k in res_keys}

In [3]:
res_dict_str

{'1gpu': [],
 'accum_steps_1': [],
 'accum_steps_2': [],
 'accum_steps_4': [],
 'accum_steps_8': [],
 'accum_steps_16': [],
 'nosync_steps_1': [],
 'nosync_steps_2': [],
 'nosync_steps_4': [],
 'nosync_steps_8': [],
 'nosync_steps_16': []}

In [4]:
import json

for train_qty in [4000, 40000]:
    for rk in res_keys:
        for seed in range(SEED_QTY):
            tmp = json.load(open(f'{OUTPUT_PREF}{train_qty}_{rk}/{seed}/all_results.json'))
            if seed == 0:
                one_res = tmp
            else:
                assert one_res.keys() == tmp.keys()
                for mk in tmp.keys():
                    one_res[mk] += tmp[mk]
                 
        for mk in one_res.keys():
            one_res[mk] /= float(SEED_QTY)
        res_dict[rk].append(one_res)
    

In [5]:
tmp

{'eval_exact_match': 78.87417218543047,
 'eval_f1': 87.17069418375785,
 'eval_train_time': 1367.486189365387}

In [6]:
for cidx, train_qty in enumerate([4000, 40000]):
    for rk in res_keys:
        speed_up = round(res_dict['1gpu'][cidx]['eval_train_time'] / res_dict[rk][cidx]['eval_train_time'], 1)
        f1 = round(res_dict[rk][cidx]['eval_f1'], 1)
        print(rk, speed_up, f1)
        if rk != '1gpu':
            res_dict_str[rk].append(f'f1={f1} {speed_up}x')
        else:
            res_dict_str[rk].append(f'f1={f1}')
    

1gpu 1.0 79.3
accum_steps_1 2.6 77.8
accum_steps_2 2.7 74.7
accum_steps_4 2.7 70.6
accum_steps_8 2.9 54.8
accum_steps_16 3.1 15.9
nosync_steps_1 2.6 77.4
nosync_steps_2 2.9 74.5
nosync_steps_4 3.3 71.9
nosync_steps_8 3.5 72.8
nosync_steps_16 3.6 74.2
1gpu 1.0 89.2
accum_steps_1 2.4 88.6
accum_steps_2 2.5 88.2
accum_steps_4 2.6 87.5
accum_steps_8 2.6 86.7
accum_steps_16 2.6 84.4
nosync_steps_1 2.4 88.8
nosync_steps_2 2.8 87.2
nosync_steps_4 3.2 87.3
nosync_steps_8 3.4 87.4
nosync_steps_16 3.6 87.3


In [7]:
res_dict_str

{'1gpu': ['f1=79.3', 'f1=89.2'],
 'accum_steps_1': ['f1=77.8 2.6x', 'f1=88.6 2.4x'],
 'accum_steps_2': ['f1=74.7 2.7x', 'f1=88.2 2.5x'],
 'accum_steps_4': ['f1=70.6 2.7x', 'f1=87.5 2.6x'],
 'accum_steps_8': ['f1=54.8 2.9x', 'f1=86.7 2.6x'],
 'accum_steps_16': ['f1=15.9 3.1x', 'f1=84.4 2.6x'],
 'nosync_steps_1': ['f1=77.4 2.6x', 'f1=88.8 2.4x'],
 'nosync_steps_2': ['f1=74.5 2.9x', 'f1=87.2 2.8x'],
 'nosync_steps_4': ['f1=71.9 3.3x', 'f1=87.3 3.2x'],
 'nosync_steps_8': ['f1=72.8 3.5x', 'f1=87.4 3.4x'],
 'nosync_steps_16': ['f1=74.2 3.6x', 'f1=87.3 3.6x']}

In [8]:
import pandas as pd
df=pd.DataFrame.from_dict(res_dict_str)

In [9]:
df

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16
0,f1=79.3,f1=77.8 2.6x,f1=74.7 2.7x,f1=70.6 2.7x,f1=54.8 2.9x,f1=15.9 3.1x,f1=77.4 2.6x,f1=74.5 2.9x,f1=71.9 3.3x,f1=72.8 3.5x,f1=74.2 3.6x
1,f1=89.2,f1=88.6 2.4x,f1=88.2 2.5x,f1=87.5 2.6x,f1=86.7 2.6x,f1=84.4 2.6x,f1=88.8 2.4x,f1=87.2 2.8x,f1=87.3 3.2x,f1=87.4 3.4x,f1=87.3 3.6x


In [10]:
df.to_csv('results.tsv')

In [11]:
!cat results.tsv

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16
0,f1=79.3,f1=77.8 2.6x,f1=74.7 2.7x,f1=70.6 2.7x,f1=54.8 2.9x,f1=15.9 3.1x,f1=77.4 2.6x,f1=74.5 2.9x,f1=71.9 3.3x,f1=72.8 3.5x,f1=74.2 3.6x
1,f1=89.2,f1=88.6 2.4x,f1=88.2 2.5x,f1=87.5 2.6x,f1=86.7 2.6x,f1=84.4 2.6x,f1=88.8 2.4x,f1=87.2 2.8x,f1=87.3 3.2x,f1=87.4 3.4x,f1=87.3 3.6x
